In [58]:
import numpy as np
import plotly.graph_objects as go
from evo.tools import file_interface

from orbslam_processor import ORBSLAMResults, ORBSLAMProcessor
from plotting_utils import create_scattermapbox, create_map_fig
from os import path

TRAJECTORY_NAME = 'vabadussild'
# TRAJECTORY_NAME = 'vabadusepst'
# TRAJECTORY_NAME = 'oldtown'

processor = ORBSLAMProcessor(path.join('~/orbslam-out', TRAJECTORY_NAME))


In [59]:
processor.initialize_model(overwrite=True)

Trial 254 Complete [00h 00m 02s]
val_loss: 0.02123873680830002

Best val_loss So Far: 0.0076425704173743725
Total elapsed time: 00h 03m 54s
INFO:tensorflow:Oracle triggered exit
Epoch 1/200
17/17 [==============================] - 0s 5ms/step - loss: 0.0501 - val_loss: 0.0339 - lr: 0.0010
Epoch 2/200
17/17 [==============================] - 0s 2ms/step - loss: 0.0200 - val_loss: 0.0201 - lr: 0.0010
Epoch 3/200
17/17 [==============================] - 0s 2ms/step - loss: 0.0126 - val_loss: 0.0142 - lr: 0.0010
Epoch 4/200
17/17 [==============================] - 0s 2ms/step - loss: 0.0094 - val_loss: 0.0121 - lr: 0.0010
Epoch 5/200
17/17 [==============================] - 0s 2ms/step - loss: 0.0076 - val_loss: 0.0114 - lr: 0.0010
Epoch 6/200
17/17 [==============================] - 0s 2ms/step - loss: 0.0068 - val_loss: 0.0098 - lr: 0.0010
Epoch 7/200
17/17 [==============================] - 0s 2ms/step - loss: 0.0061 - val_loss: 0.0100 - lr: 0.0010
Epoch 8/200
17/17 [===================

In [60]:
processor.fit_trajectories()


13/13 [==============================] - 0s 403us/step


In [61]:
processor.model.best_hps.values

{'il_units': 56,
 'il_activation': 'tanh',
 'num_hidden_layers': 4,
 'huber_delta': 0.1,
 'hl_0_units': 58,
 'hl_0_activation': 'relu',
 'hl_0_is_regularizer': True,
 'hl_1_units': 41,
 'hl_1_activation': 'tanh',
 'hl_1_is_regularizer': True,
 'batch_size': 32,
 'hl_0_regularizer': 'l1',
 'hl_2_units': 27,
 'hl_2_activation': 'tanh',
 'hl_2_is_regularizer': False,
 'hl_1_regularizer': 'l2',
 'hl_2_regularizer': 'l2',
 'hl_3_units': 29,
 'hl_3_activation': 'relu',
 'hl_3_is_regularizer': False,
 'hl_4_units': 26,
 'hl_4_activation': 'tanh',
 'hl_4_is_regularizer': False,
 'hl_3_regularizer': 'l1',
 'hl_4_regularizer': 'l1',
 'tuner/epochs': 210,
 'tuner/initial_epoch': 70,
 'tuner/bracket': 3,
 'tuner/round': 3,
 'tuner/trial_id': '0204'}

In [62]:
processor.model.model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_5 (Dense)             (None, 56)                224       
                                                                 
 dense_6 (Dense)             (None, 58)                3306      
                                                                 
 dense_7 (Dense)             (None, 41)                2419      
                                                                 
 dense_8 (Dense)             (None, 27)                1134      
                                                                 
 dense_9 (Dense)             (None, 29)                812       
                                                                 
 dense_10 (Dense)            (None, 3)                 90        
                                                                 
Total params: 7,985
Trainable params: 7,985
Non-traina

In [63]:
processor.save_trajectories()

In [64]:
fig_2d = processor.create_2d_plot_utm()
fig_2d.show()

In [65]:
fig_slam = processor.create_2d_plot_slam()
fig_slam.show()

In [66]:
fig_map = processor.create_map_plot()
fig_map.show()

In [67]:

from utils import create_training_splits


training_data = (processor.orbslam.mapping.trajectory, processor.orbslam.mapping.trajectory_gt_utm)
extra_data = (processor.orbslam.localization[0].trajectory, processor.orbslam.localization[0].trajectory_gt_utm)

training, validation, testing = create_training_splits(training_data, extra_data, 0.20)


In [68]:
fig_training = go.Figure()

fig_training.add_trace(go.Scatter(
    x=training[0][:, 0],
    y=training[0][:, 2],
    mode='markers',
    name="training"
))
fig_training.add_trace(go.Scatter(
    x=validation[0][:, 0],
    y=validation[0][:, 2],
    mode='markers',
    name="validation"
))
fig_training.add_trace(go.Scatter(
    x=testing[0][:, 0],
    y=testing[0][:, 2],
    mode='markers',
    name="testing"
))

fig_training.update_layout(
    yaxis=dict(scaleanchor="x", scaleratio=1),
    height=1200
)

print(f'training: {training[0].shape}')
print(f'validation: {validation[0].shape}')
print(f'testing: {testing[0].shape}')

fig_training.show()

training: (529, 3)
validation: (66, 3)
testing: (66, 3)
